### Dataframes

In [1]:
from collections import Counter
import pandas as pd
import os
import datetime
from instances import Instance

In [2]:
inst= Instance()

In [3]:
inst.init_order()

In [4]:
inst.init_tickets()

In [7]:
inst.init_depot()
inst.init_employee()
inst.init_depot_loc()

In [6]:
inst.init_dist()
inst.init_time()

In [8]:
inst.clean_depot()
inst.clean_employee()

Clean Depot
Clean employee


In [9]:
inst.clean_tickets()

In [134]:
ticket_date = list(dict.fromkeys(inst.df_tickets['DATE']))
# print(ticket_date)
# ticket_date = [ datetime.date(2020, 10, 10)]
travel = dict()
for jour in ticket_date:
    day = jour.day
    if day < 10:
        day = f"0{day}"
    df_day_orders = inst.df_orders[inst.df_orders['DATE'] == jour]
    df_day_orders = df_day_orders[df_day_orders['JOB_DESC'] != 'PICKUP']
    list_ordres = list(set(df_day_orders['ORDER_ID']))
    drivers = list(
        set(inst.df_tickets[inst.df_tickets['ORDER_ID'].isin(list_ordres)]['DRIVER_NBR']))
    df_tickets = inst.df_tickets[inst.df_tickets['ORDER_ID'].isin(list_ordres)]
    # print(drivers)
    if 1 in drivers:
        continue
    print(jour)
    all_dist1, all_dist2 = 0, 0
    all_temps1, all_temp2 = 0, 0
    demand = 0
    for drv in drivers:
        # print(drv)
        drv_tickets = df_tickets[df_tickets['DRIVER_NBR'] == drv]
        drv_tickets = drv_tickets[drv_tickets['PROD_NBR'] == 'Z']
        usine = Counter(drv_tickets['SHIP_LOC']).most_common(1)[0][0]
        drv_tickets.sort_values(["BEGIN_LOAD"], inplace=True)

        # print((df_day_orders[df_day_orders['ORD']].columns))
        dist1, dist2 = 0, 0
        t1, t2 = 0, 0
        prev = inst.df_depots.loc[usine]['Location ID']
        _next = 0
        for i, index in enumerate(drv_tickets.index):
            cur = drv_tickets.loc[index]
            # print(cur)
            dock = inst.df_depots.loc[cur['SHIP_LOC']]['Location ID']
            pos = df_day_orders[df_day_orders['ORDER_ID']
                                == cur['ORDER_ID']]['LOCATION ID'].max()
            dist1 = dist1 + inst.dist[prev][dock] + inst.dist[dock][pos]
            t1 = t1 + inst.temps[prev][dock] + inst.temps[dock][pos]
            prev = pos
            demand = demand + cur['QUANTITY']
            # print(dock,pos, dist)

        dist1 = dist1 + \
            inst.dist[prev][inst.df_depots.loc[usine]['Location ID']]
        t1 = t1 + inst.temps[prev][inst.df_depots.loc[usine]['Location ID']]
        # print('dist ', dist)
        all_dist1 = all_dist1 + dist1
        all_temps1 = all_temps1 + t1
    # print('all_dist ', all_dist1,all_temps1,demand)
    travel[f"{jour.year}{jour.month}{day}"] = [demand,all_dist1]

df = pd.DataFrame.from_dict(travel, orient='index')
df.reset_index(inplace=True)
df.to_csv(
    'travel.csv', sep=';', decimal='.', header=['Name','Demand','D'])

2020-10-10
2020-10-17
2020-10-21
2020-10-26
2020-10-31
2020-11-07
2020-11-09
2020-11-21


In [14]:
inst.df_orders.loc[:,jour]

KeyError: datetime.date(2020, 10, 10)

In [7]:
df_depots = pd.read_excel(f"adresses_complete.xlsx",engine='openpyxl')
df_depots = df_depots[df_depots['Location_type'] != 'customer']
df_depots.dropna(axis=1,  inplace=True)
df_depots['NBR'] = df_depots['JOB_DESC'].apply(
    lambda x: x if len(x.split(' ')) == 1 else int(x.split(' ')[1]))
df_depots.set_index('NBR',inplace=True)
df_depots.drop(['Location_type', 'JOB_DESC', 'MAP_PAGE', 
       'Postal_code', 'JOBSTATE', 'LATITUDE_Y', 'LONGITUDE_X'],axis=1,inplace=True)

In [8]:
def beginTime(week,day,driver):
    week_op = df_tickets[df_tickets['WEEK']==week]
    day_op = week_op[week_op['DATE']==day]
    _data = day_op[day_op['DRIVER_NBR']==driver][["BEGIN_LOAD","ARRIVE_PLANT"]]
    _data.sort_values(["BEGIN_LOAD"],inplace=True)
   
    return f"{_data.iloc[0,0].time()}".split('.')[0]

def endTime(week,day,driver):
    week_op = df_tickets[df_tickets['WEEK']==week]
    day_op = week_op[week_op['DATE']==day]
    _data = day_op[day_op['DRIVER_NBR']==driver][["BEGIN_LOAD","ARRIVE_PLANT"]]
    _data.sort_values(["BEGIN_LOAD"],inplace=True)
    return f"{_data.iloc[-1,1].time()}".split('.')[0]

In [13]:
#%% Clean data
df_tickets.dropna(axis=0,inplace=True)

df_tickets['CUST'] = df_tickets['ORDER_ID'].apply(lambda x: df_orders[df_orders['ORDER_ID']==x]['CUST_NBR'].max())
df_tickets['JOB_DESC'] = df_tickets['ORDER_ID'].apply(lambda x: df_orders[df_orders['ORDER_ID']==x]['JOB_DESC'].max())
                                                      
df_tickets['DATE'] = df_tickets['TICKET_DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
df_tickets['WEEK'] = df_tickets['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).isocalendar()[1])
df_tickets['DAY'] = df_tickets['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).toordinal())

df_tickets = df_tickets[df_tickets['SHIP_LOC']!=99]
df_tickets = df_tickets[df_tickets['JOB_DESC'] != 'PICKUP']

# df_tickets = df_tickets[df_tickets['PROD_NBR']=='Z']

df_tickets.drop(['DTICKETHIS_TICKET_ID',  'TICKET_NBR', 'TICKET_DATE',
       'TICKET_TIME',   'BEGIN_POUR', 'FINISH_POUR',   'FINISH_LOAD',
       'TO_PLANT', 'ON_JOB', 'TO_JOB', 'LOAD', 'LOADNR',
       'DTICKETPRODUCTHIS_TICKET_ID',   'CATEGORY_CODE'],axis=1,inplace=True)


In [15]:
ticket_date = list(dict.fromkeys(df_tickets['DATE']))
print(ticket_date)
ticket_date = [ datetime.date(2020, 10, 10)]
for jour in ticket_date:
    list_ordres = list(set( df_orders[df_orders['DATE']==jour][df_orders['JOB_DESC']!='PICKUP']['ORDER_ID']))
    print(list_ordres)
    pass

[datetime.date(2020, 10, 1), datetime.date(2020, 10, 2), datetime.date(2020, 10, 3), datetime.date(2020, 10, 5), datetime.date(2020, 10, 6), datetime.date(2020, 10, 7), datetime.date(2020, 10, 8), datetime.date(2020, 10, 9), datetime.date(2020, 10, 10), datetime.date(2020, 10, 13), datetime.date(2020, 10, 14), datetime.date(2020, 10, 15), datetime.date(2020, 10, 16), datetime.date(2020, 10, 17), datetime.date(2020, 10, 19), datetime.date(2020, 10, 20), datetime.date(2020, 10, 21), datetime.date(2020, 10, 22), datetime.date(2020, 10, 23), datetime.date(2020, 10, 24), datetime.date(2020, 10, 26), datetime.date(2020, 10, 27), datetime.date(2020, 10, 28), datetime.date(2020, 10, 29), datetime.date(2020, 10, 30), datetime.date(2020, 10, 31), datetime.date(2020, 11, 2), datetime.date(2020, 11, 3), datetime.date(2020, 11, 4), datetime.date(2020, 11, 5), datetime.date(2020, 11, 6), datetime.date(2020, 11, 7), datetime.date(2020, 11, 9), datetime.date(2020, 11, 10), datetime.date(2020, 11, 11),

KeyError: 'DATE'

In [10]:
df_tickets["BEGIN"] = df_tickets.apply(lambda x: beginTime(x.WEEK,x.DATE,x.DRIVER_NBR),axis=1)
df_tickets["END"] = df_tickets.apply(lambda x: endTime(x.WEEK,x.DATE,x.DRIVER_NBR),axis=1)
df_tickets.head()

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,PROD_NBR,QUANTITY,CUST,JOB_DESC,DATE,WEEK,DAY,BEGIN,END
0,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,3510RPC1,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:38:00,16:34:16
1,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,USPC120,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:38:00,16:34:16
2,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,P10,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:38:00,16:34:16
3,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,SC,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:38:00,16:34:16
4,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,Z,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:38:00,16:34:16


In [14]:
# df_orders['DATE'] = df_orders['SHIPDATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
# df_orders['WEEK'] = df_orders['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).isocalendar()[1])
# df_orders['DAY'] = df_orders['DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day).toordinal())

df_orders = df_orders[df_orders['JOB_DESC'] != 'PICKUP']

df_orders.drop(['COMP_NBR',  'ORDER_STATUS',
       'ORDER_DATE', 'ORDER_TIME (min)', 'SHIPDATE', 'Day_of_the_week',
       'SHIPTIME (min)', 'ZONE_NBR', 'WORK_TYPE', 'SCHED_LOC',
       'JOB_DESC', 'ORDER_DESC', 'MAP_PAGE', 'JOBCITY', 'Postal code ',
       'JOBSTATE', 'LATITUDE_Y', 'LONGITUDE_X', 'TRAVEL_MINUTES',
       'UNLOAD_MINUTES', 'QTY_PER_HOUR', 'TRUCK_REQUEST',
       'CATEGORY_CODE', 'UNIT_OF_MEASURE', 'QUANTITY', 'LOOKUP_DESC',
       'CONFIG_DESC', 'Unnamed: 33', 'Unnamed: 34', 'Location ',
       'ID'],axis=1,inplace=True)
# df_orders.drop_duplicates(inplace=True)

KeyError: 'JOB_DESC'

In [15]:
df_tickets.reset_index()
df=pd.merge(left=df_tickets,right= df_orders,how="inner",on=['ORDER_ID','PROD_NBR'])
# df = df[df['DRIVER_NBR']!=1]
df.head()

,SHIP_LOC,ORDER_ID,DRIVER_NBR,BEGIN_LOAD,ARRIVE_PLANT,TRUCK_NBR,SCHED_LOC,PROD_NBR,QUANTITY,CUST,JOB_DESC,DATE,WEEK,DAY,BEGIN,END,ORDER_LOC,ORDER_NBR,CUST_NBR,LOCATION ID
0,54,917231,15083.0,2020-10-01 14:16:23.720,2020-10-01 16:34:16.832,460,54,3510RPC1,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:38:00,16:34:16,0,603592,104590,0
1,54,917231,15143.0,2020-10-01 15:00:56.091,2020-10-01 16:34:14.566,370,54,3510RPC1,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:24:38,16:34:14,0,603592,104590,0
2,54,917231,15070.0,2020-10-01 15:20:11.886,2020-10-01 17:10:50.250,426,54,3510RPC1,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,06:57:28,17:10:50,0,603592,104590,0
3,54,917231,15507.0,2020-10-01 15:59:26.468,2020-10-01 18:21:08.017,384,54,3510RPC1,8.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,06:46:49,18:21:08,0,603592,104590,0
4,54,917231,7000059.0,2020-10-01 16:15:50.000,2020-10-01 18:35:24.000,019,54,3510RPC1,7.0,104590,261 RUE QUEEN (ET KING),2020-10-01,40,737699,07:05:27,18:35:24,0,603592,104590,0


### Ajouter Id  dans la matrice de distance des dépots

In [16]:
df['SHIP_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.SHIP_LOC]['Location ID'],axis=1)
df['SCHED_LOC_ID'] = df.apply(lambda x: df_depots.loc[x.SCHED_LOC]['Location ID'],axis=1)

### Ajouter le temps parcouru aller-retour par visite

In [17]:
df['t1']=df.apply(lambda x: (temps[x.SHIP_LOC_ID][x['LOCATION ID']]+temps[x['LOCATION ID']][x.SHIP_LOC_ID])/60,axis=1)
df['t2']=df.apply(lambda x: (temps[x.SCHED_LOC_ID][x['LOCATION ID']]+temps[x['LOCATION ID']][x.SCHED_LOC_ID])/60,axis=1)

### Ajouter la distance parcourue aller-retour par visite

In [18]:
df['d1']=df.apply(lambda x: (dist[x.SHIP_LOC_ID][x['LOCATION ID']]+dist[x['LOCATION ID']][x.SHIP_LOC_ID]),axis=1)
df['d2']=df.apply(lambda x: (dist[x.SCHED_LOC_ID][x['LOCATION ID']]+dist[x['LOCATION ID']][x.SCHED_LOC_ID]),axis=1)

In [19]:
df.reset_index(inplace=True)
df.set_index(['WEEK'],inplace=True)
df.sort_index(inplace=True)
# df = df.loc[[41, 43, 44, 45, 46, 47, 48]]

In [21]:
# data with distance
df.reset_index(inplace=True)
df.set_index(['WEEK','DATE','TRUCK_NBR','DRIVER_NBR','ORDER_ID'],inplace=True)
df.sort_index(inplace=True)
df.groupby(by=['WEEK','DATE','TRUCK_NBR','DRIVER_NBR','ORDER_ID','BEGIN','END']).agg({'t1':
    'sum','t2':'sum','QUANTITY':'sum'}).to_csv('schedule_dist_quant.csv',sep=';',decimal=',')


In [33]:
# data with distance
df.reset_index(inplace=True)
df.set_index(['WEEK','DATE','ORDER_ID'],inplace=True)
df.sort_index(inplace=True)

df.groupby(by=['WEEK','DATE','ORDER_ID','PROD_NBR','CUST_NBR']).agg({'t1':'sum','t2':'sum','QUANTITY':'sum',
'DRIVER_NBR':'count','TRUCK_NBR':'count'}).to_csv('schedule_order_quant.csv',sep=';',decimal=',')
df.groupby(by=['WEEK','DATE']).agg({'t1':'sum','t2':'sum','d1':'sum','d2':'sum','QUANTITY':'sum',
                                    'DRIVER_NBR':'nunique','TRUCK_NBR':'nunique'}).to_csv('stat_by_day.csv',sep=';',decimal=',')
df.head()

TRUCK_NBR  DRIVER_NBR  index  SHIP_LOC  \
WEEK DATE       ORDER_ID                                          
40   2020-10-01 917231         019   7000059.0      4        54   
                917231         370     15143.0      1        54   
                917231         384     15507.0      3        54   
                917231         426     15070.0      2        54   
                917231         460     15083.0      0        54   

                                      BEGIN_LOAD            ARRIVE_PLANT  \
WEEK DATE       ORDER_ID                                                   
40   2020-10-01 917231   2020-10-01 16:15:50.000 2020-10-01 18:35:24.000   
                917231   2020-10-01 15:00:56.091 2020-10-01 16:34:14.566   
                917231   2020-10-01 15:59:26.468 2020-10-01 18:21:08.017   
                917231   2020-10-01 15:20:11.886 2020-10-01 17:10:50.250   
                917231   2020-10-01 14:16:23.720 2020-10-01 16:34:16.832   

                          SCHED_LOC  PROD_NBR  QUANTITY    CUST  ...  \
WEEK DATE       ORDER_ID                                         ...   
40   2020-10-01 917231           54  3510RPC1       7.0  104590  ...   
                917231           54  3510RPC1       8.0  104590  ...   
                917231           54  3510RPC1       8.0  104590  ...   
                917231           54  3510RPC1       8.0  104590  ...   
                917231           54  3510RPC1       8.0  104590  ...   

                         ORDER_LOC  ORDER_NBR CUST_NBR LOCATION ID  \
WEEK DATE       ORDER_ID                                             
40   2020-10-01 917231           0     603592   104590           0   
                917231           0     603592   104590           0   
                917231           0     603592   104590           0   
                917231           0     603592   104590           0   
                917231           0     603592   104590           0   

                          SHIP_LOC_ID  SCHED_LOC_ID     t1     t2      d1  \
WEEK DATE       ORDER_ID                                                    
40   2020-10-01 917231           2494          2494  4.235  4.235  2.4441   
                917231           2494          2494  4.235  4.235  2.4441   
                917231           2494          2494  4.235  4.235  2.4441   
                917231           2494          2494  4.235  4.235  2.4441   
                917231           2494          2494  4.235  4.235  2.4441   

                              d2  
WEEK DATE       ORDER_ID          
40   2020-10-01 917231    2.4441  
                917231    2.4441  
                917231    2.4441  
                917231    2.4441  
                917231    2.4441  

[5 rows x 24 columns]

In [ ]:
df.groupby(by=['DATE','WEEK','ORDER_ID']).agg({'t1':'sum','t2':'sum'})

In [ ]:
df.groupby(by=['DATE','DRIVER_NBR','ORDER_ID']).agg({'t1':'sum','t2':'sum'})


In [ ]:
df.head()

In [ ]:
df1= df.copy()
df1.reset_index(inplace=True)
df1.set_index('DATE')
df1.groupby(by=['DATE']).agg({'t1':'sum','t2':'sum','QUANTITY':'sum','TRUCK_NBR':'nunique',
                              'SHIP_LOC':'nunique','ORDER_ID':'nunique'}).to_csv("CQ.csv",sep=';',decimal=',')

In [ ]:
t=df.groupby(by=['DATE','WEEK','DRIVER_NBR']).agg({'t1':'sum','t2':'sum'})
# t.to_csv('driver.csv')
# t.xs(('2020-11-16',47)).sum()

In [ ]:
d= df.groupby(by=['DATE','ORDER_ID','DRIVER_NBR']).agg({'t1':'sum','t2':'sum'})
# d.xs(("2020-10-30"))
print(d.xs((930040),level=1)/60)
print(d.xs((930040),level=1).sum()/60)

# df['DATE'].unique()